In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

     |████████████████████████████████| 4.3MB 2.6MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [3]:
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [2]:
cd 'drive/My Drive/Colab Notebooks/matrix_dw/matrix_two/dw_matrix_car'

/content/drive/My Drive/Colab Notebooks/matrix_dw/matrix_two/dw_matrix_car


In [0]:
df = pd.read_hdf('data/car.h5')

In [0]:
Suffix_cat = '_cat'
for feat in df.columns:
  if isinstance(df[feat][0], list):continue
  
  factorized_values = df[feat].factorize()[0]
  if Suffix_cat in feat:
     df[feat] = factorized_values
  else:   
    df[feat + Suffix_cat] = df[feat].factorize()[0]

In [6]:
cat_feats = [x for x in df.columns if Suffix_cat in x ]
cat_feats = [x for x in cat_feats if 'price' not in x ]
len(cat_feats)


151

In [0]:
def run_model(model, feats):

  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [13]:
df[ 'param_rok-produkcji' ] = df[ 'param_rok-produkcji' ].map(lambda x: -1 if str(x) == 'None' else int(x))
df ['param_moc'] = df ['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]) )
df ['param_pojemność-skokowa'] = df ['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ','')) )


AttributeError: ignored

##Hyperopt

In [0]:
feats = [
'param_napęd_cat',
'param_rok-produkcji',
'param_stan_cat',
'param_skrzynia-biegów_cat',
'param_faktura-vat_cat',
'param_moc',
'param_marka-pojazdu_cat',
'feature_kamera-cofania_cat',
'param_typ_cat',
'param_pojemność-skokowa',
'seller_name_cat',
'feature_wspomaganie-kierownicy_cat',
'param_model-pojazdu_cat',
'param_wersja_cat',
'param_kod-silnika_cat',
'feature_system-start-stop_cat',
'feature_asystent-pasa-ruchu_cat',
'feature_czujniki-parkowania-przednie_cat',
'feature_łopatki-zmiany-biegów_cat',
'feature_regulowane-zawieszenie_cat']

In [0]:
def obj_func(params):
  print("Training with params: ")
  print(params)

  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)

  return {'loss' : np.abs(mean_mae), 'status' : STATUS_OK}

In [18]:
#space

xgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate', np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth', np.arange(5, 16, 1, dtype=int)),
    'subsample':        hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'objective':        'reg:squarederror',    
    'n_estimators':     100,
    'seed':             0,
}

#run

best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)

Training with params: 
{'colsample_bytree': 0.5, 'learning_rate': 0.3, 'max_depth': 9, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.75}
Training with params: 
{'colsample_bytree': 0.9, 'learning_rate': 0.25, 'max_depth': 7, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.75}
Training with params: 
{'colsample_bytree': 0.65, 'learning_rate': 0.1, 'max_depth': 13, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params: 
{'colsample_bytree': 0.75, 'learning_rate': 0.2, 'max_depth': 14, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.6000000000000001}
Training with params: 
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.5}
Training with params: 
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.2, 'max_depth': 12, 'n_estima